In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import pickle

# Dữ liệu câu hỏi và câu trả lời
data = [
    ("Giờ làm việc của công ty là gì?", "Công ty chúng tôi làm việc từ 8:00 sáng đến 5:00 chiều, từ thứ Hai đến thứ Sáu."),
    ("Địa chỉ của công ty là gì?", "Công ty chúng tôi ở số 123 Đường ABC, Quận XYZ, Thành phố HCM."),
    ("Làm thế nào để liên hệ với bộ phận hỗ trợ?", "Bạn có thể liên hệ với bộ phận hỗ trợ qua email support@company.com hoặc gọi số 0123456789."),
    ("Công ty có cung cấp dịch vụ giao hàng không?", "Có, chúng tôi cung cấp dịch vụ giao hàng toàn quốc."),
    ("Chính sách đổi trả hàng của công ty là gì?", "Bạn có thể đổi trả hàng trong vòng 30 ngày kể từ ngày mua hàng."),
    ("Giá sản phẩm X là bao nhiêu?", "Giá của sản phẩm X là 1,000,000 VND."),
    ("Công ty có chương trình khuyến mãi nào hiện tại không?", "Hiện tại, chúng tôi có chương trình giảm giá 20% cho tất cả các sản phẩm."),
    ("Làm sao để đăng ký nhận tin tức từ công ty?", "Bạn có thể đăng ký nhận tin tức từ công ty bằng cách nhập email vào form trên trang chủ của chúng tôi."),
    ("Công ty có chi nhánh ở đâu?", "Công ty chúng tôi có chi nhánh tại Hà Nội, Đà Nẵng và Thành phố Hồ Chí Minh."),
    ("Thời gian giao hàng là bao lâu?", "Thời gian giao hàng là từ 3-5 ngày làm việc."),
    ("Chính sách bảo mật của công ty là gì?", "Chính sách bảo mật của chúng tôi có thể được xem tại trang chính sách bảo mật trên website."),
    ("Làm sao để theo dõi đơn hàng của tôi?", "Bạn có thể theo dõi đơn hàng của mình bằng cách nhập mã đơn hàng trên trang theo dõi đơn hàng."),
    ("Sản phẩm này có bảo hành không?", "Có, sản phẩm này có bảo hành 12 tháng."),
    ("Làm sao để yêu cầu hỗ trợ kỹ thuật?", "Bạn có thể yêu cầu hỗ trợ kỹ thuật bằng cách gửi email đến support@company.com."),
    ("Công ty có hỗ trợ thanh toán trả góp không?", "Có, chúng tôi hỗ trợ thanh toán trả góp qua thẻ tín dụng."),
    ("Tôi có thể hủy đơn hàng của mình không?", "Bạn có thể hủy đơn hàng nếu đơn hàng chưa được xử lý."),
    ("Phí vận chuyển là bao nhiêu?", "Phí vận chuyển là 50,000 VND cho mỗi đơn hàng."),
    ("Sản phẩm này có sẵn không?", "Sản phẩm này hiện đang có sẵn."),
    ("Chính sách bảo hành của công ty là gì?", "Chính sách bảo hành của chúng tôi có thể được xem tại trang chính sách bảo hành trên website."),
    ("Tôi có thể mua sản phẩm này ở đâu?", "Bạn có thể mua sản phẩm này trên website của chúng tôi hoặc tại các cửa hàng chi nhánh."),
    ("Công ty có dịch vụ lắp đặt không?", "Có, chúng tôi có dịch vụ lắp đặt cho các sản phẩm."),
    ("Phương thức thanh toán của công ty là gì?", "Chúng tôi chấp nhận thanh toán bằng thẻ tín dụng, chuyển khoản ngân hàng và tiền mặt khi giao hàng."),
    ("Công ty có hỗ trợ đổi sản phẩm không?", "Có, chúng tôi hỗ trợ đổi sản phẩm trong vòng 30 ngày kể từ ngày mua hàng."),
    ("Làm sao để tạo tài khoản trên website?", "Bạn có thể tạo tài khoản bằng cách nhấp vào nút 'Đăng ký' trên trang chủ và điền thông tin cần thiết."),
    ("Công ty có nhận đơn hàng quốc tế không?", "Hiện tại chúng tôi chỉ nhận đơn hàng trong nước."),
    ("Làm sao để hủy đăng ký nhận tin tức?", "Bạn có thể hủy đăng ký nhận tin tức bằng cách nhấp vào liên kết 'Hủy đăng ký' ở cuối email chúng tôi gửi."),
    ("Chính sách bảo hành của sản phẩm X là gì?", "Sản phẩm X có bảo hành 12 tháng."),
    ("Công ty có dịch vụ tư vấn miễn phí không?", "Có, chúng tôi cung cấp dịch vụ tư vấn miễn phí qua điện thoại và email."),
    ("Làm sao để xem lịch sử mua hàng của tôi?", "Bạn có thể xem lịch sử mua hàng bằng cách đăng nhập vào tài khoản của mình và chọn 'Lịch sử mua hàng'."),
    ("Sản phẩm Y có còn hàng không?", "Sản phẩm Y hiện còn hàng."),
    ("Tôi có thể yêu cầu xuất hóa đơn không?", "Có, bạn có thể yêu cầu xuất hóa đơn khi thanh toán."),
    ("Chính sách vận chuyển của công ty là gì?", "Chính sách vận chuyển của chúng tôi có thể được xem tại trang chính sách vận chuyển trên website."),
    ("Làm sao để liên hệ với bộ phận chăm sóc khách hàng?", "Bạn có thể liên hệ với bộ phận chăm sóc khách hàng qua email support@company.com hoặc gọi số 0123456789."),
    ("Công ty có chương trình khách hàng thân thiết không?", "Có, chúng tôi có chương trình khách hàng thân thiết với nhiều ưu đãi."),
    ("Làm sao để sử dụng mã giảm giá?", "Bạn có thể nhập mã giảm giá khi thanh toán để nhận ưu đãi."),
    ("Sản phẩm này có phù hợp cho trẻ em không?", "Sản phẩm này phù hợp cho trẻ em từ 3 tuổi trở lên."),
    ("Làm sao để biết thêm thông tin về sản phẩm?", "Bạn có thể xem thêm thông tin về sản phẩm trên trang chi tiết sản phẩm."),
    ("Công ty có chính sách hỗ trợ doanh nghiệp không?", "Có, chúng tôi có chính sách hỗ trợ đặc biệt cho các doanh nghiệp."),
    ("Làm sao để thay đổi địa chỉ giao hàng?", "Bạn có thể thay đổi địa chỉ giao hàng bằng cách liên hệ với bộ phận chăm sóc khách hàng."),
    ("Sản phẩm này có phải hàng chính hãng không?", "Có, tất cả các sản phẩm của chúng tôi đều là hàng chính hãng."),
    ("Làm sao để biết tình trạng đơn hàng của tôi?", "Bạn có thể theo dõi tình trạng đơn hàng bằng cách nhập mã đơn hàng trên trang theo dõi đơn hàng."),
    ("Chính sách hoàn tiền của công ty là gì?", "Chính sách hoàn tiền của chúng tôi có thể được xem tại trang chính sách hoàn tiền trên website."),
    ("Công ty có hỗ trợ kỹ thuật không?", "Có, chúng tôi có đội ngũ kỹ thuật sẵn sàng hỗ trợ bạn."),
    ("Làm sao để nhận hóa đơn điện tử?", "Bạn có thể yêu cầu nhận hóa đơn điện tử khi thanh toán."),
    ("Công ty có bán lẻ sản phẩm không?", "Có, chúng tôi bán lẻ tất cả các sản phẩm."),
    ("Làm sao để yêu cầu bảo hành sản phẩm?", "Bạn có thể yêu cầu bảo hành sản phẩm bằng cách liên hệ với bộ phận chăm sóc khách hàng."),
    ("Công ty có hỗ trợ giao hàng nhanh không?", "Có, chúng tôi có dịch vụ giao hàng nhanh trong vòng 24 giờ."),
    ("Chính sách thanh toán của công ty là gì?", "Chính sách thanh toán của chúng tôi có thể được xem tại trang chính sách thanh toán trên website."),
    ("Làm sao để đăng nhập vào tài khoản của tôi?", "Bạn có thể đăng nhập vào tài khoản bằng cách nhấp vào nút 'Đăng nhập' trên trang chủ và nhập thông tin tài khoản."),
]

# Chia dữ liệu thành các câu hỏi và câu trả lời
questions, answers = zip(*data)

# Tạo pipeline với TfidfVectorizer và LogisticRegression
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', LogisticRegression())
])

# Huấn luyện mô hình
pipeline.fit(questions, answers)

# Lưu mô hình đã huấn luyện
with open('chatbot_model.pkl', 'wb') as file:
    pickle.dump(pipeline, file)


In [ ]:
with open('chatbot_model.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

In [ ]:
import pickle

# Tải mô hình từ file
with open('chatbot_model.pkl', 'rb') as file:
    model = pickle.load(file)


In [ ]:
def generate_answer(question):
    # Dự đoán câu trả lời
    answer = model.predict([question])[0]
    return answer

# Ví dụ sử dụng
question = "Dịa chỉ của công ty"
print(generate_answer(question))


Công ty chúng tôi ở số 123 Đường ABC, Quận XYZ, Thành phố HCM.
